In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.special import logit, expit
import statsmodels.formula.api as smf
import statsmodels.api as sm
from collections import defaultdict
from sklearn.metrics import brier_score_loss
pd.set_option('display.max_columns', 100)

In [52]:
def update_elo(abs_GD, Res, eloF, eloA, isHome, K=40, HFA=100) :
    G = np.where(abs_GD <= 1, 1, np.where(abs_GD == 2, 3/2, (11+abs_GD)/8))
    dr = (eloF - eloA) + (HFA if isHome else - HFA)
    We = 1 / (1 + (10 ** (-dr/400)))
    return eloF + K*G*(Res-We)

In [53]:
df = pd.read_csv('soccer18m.csv', index_col = False)
df = df.drop(['pD', 'pH', 'pA'], axis=1)
df = df.loc[df['Y'] > 14]

In [54]:
df['abs_GD'] = np.abs(df.FTHG - df.FTAG)
df['GD'] = (df.FTHG - df.FTAG).astype(int)
df['Res_Home'] = np.where(df['FTHG'] > df['FTAG'], 1, np.where( df['FTHG'] < df['FTAG'], 0, 0.5))
df['Res_Away'] = 1 - df.Res_Home
df['Home_elo_pre'] = 1000
df['Home_elo_post'] = 1000
df['Away_elo_pre'] = 1000
df['Away_elo_post'] = 1000

In [55]:
helopost = np.full(len(df), 1000.0)
aelopost = np.full(len(df), 1000.0)
elopost = defaultdict(float)
for i in range(len(df)) :
    curr_home = df.HomeTeam.iat[i]
    curr_away = df.AwayTeam.iat[i]
    helopost[i] = elopost[curr_home]
    if helopost[i] == 0:
        helopost[i] = 1000
    aelopost[i] = elopost[curr_away]
    if aelopost[i] == 0:
        aelopost[i] = 1000
    elopost[curr_home] = update_elo(df.abs_GD.iat[i], df.Res_Home.iat[i], helopost[i], aelopost[i], 1)
    elopost[curr_away] = update_elo(df.abs_GD.iat[i], df.Res_Away.iat[i], aelopost[i], helopost[i], 0)

In [56]:
df['Home_elo_post'] = helopost
df['Away_elo_post'] = aelopost

In [57]:
def getelo(df, i, team):
    newdf = df.head(i - 1)
    newdf = newdf[(newdf['HomeTeam'] == team) | (newdf['AwayTeam'] == team)]
    newdf = newdf.sort_values(['Date'], ascending = False)
    newdf = newdf.head(1)
#     print('yes')
    if newdf.empty:
#         print('empty')
        return 1000
#     print(newdf['HomeTeam'].values[0])
#     print(newdf['AwayTeam'].all() == team)

    if (newdf['HomeTeam'].values[0] == team):
        return newdf['Home_elo_post'].values[0]
    else:
        return newdf['Away_elo_post'].values[0]

In [58]:
for i in range(len(df)):
    curr_home = df.HomeTeam.iat[i]
    curr_away = df.AwayTeam.iat[i]
    helopre = getelo(df,i,curr_home)
    aelopre = getelo(df,i,curr_away)
    df.Home_elo_pre.iat[i] = helopre
    df.Away_elo_pre.iat[i] = aelopre
    df.Home_elo_post.iat[i] = update_elo(df.abs_GD.iat[i], df.Res_Home.iat[i],helopre, aelopre, 1)
    df.Away_elo_post.iat[i] = update_elo(df.abs_GD.iat[i], df.Res_Away.iat[i],aelopre, helopre, 0)



In [63]:
dfhw = df.loc[df['Y'] < 18]
df18 = df.loc[df['Y'] == 18]
dfhw = dfhw.sort_values(['Date'], ascending=False)

,GameID,isHome,Home_win,Y,Date,Div,Team,FTG,HTG,S,ST,xG,GD,Pts,xPts,Conversion,Res,elo_pre,elo_post
14183,8705,0,1,17,2018-05-20,Serie_A,Bologna,0,0,8,5,0.391913,-1,0,0,0.000000,0.0,895,879.815907
6873,8699,1,0,17,2018-05-20,Serie_A,Genoa,1,0,12,2,1.104570,-1,0,1,0.905330,0.0,938,920.955086
2634,3394,1,0,17,2018-05-20,La_Liga,Ath Bilbao,0,0,12,3,0.851602,-1,0,1,0.000000,0.0,1007,984.316860
9938,3394,0,0,17,2018-05-20,La_Liga,Espanol,1,1,11,4,1.087430,1,3,1,0.919599,1.0,1049,1072.831183
2635,3395,1,0,17,2018-05-20,La_Liga,Ath Madrid,2,1,8,4,1.468220,0,1,1,1.362194,0.5,1233,1219.268062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8437,1513,0,0,17,2018-05-13,EPL,Man City,1,0,13,2,0.765614,1,3,1,1.306141,1.0,1409,1413.956269
1133,1513,1,0,17,2018-05-13,EPL,Southampton,0,0,8,3,0.506624,-1,0,1,0.000000,0.0,936,932.097948
8436,1512,0,1,17,2018-05-13,EPL,Chelsea,0,0,6,2,0.843284,-3,0,0,0.000000,0.0,1171,1125.724066
1132,1512,1,1,17,2018-05-13,EPL,Newcastle,3,1,16,6,2.241970,3,3,3,1.338109,1.0,958,1004.670093


,GameID,isHome,Home_win,Y,Date,Div,Team,FTG,HTG,S,ST,xG,GD,Pts,xPts,Conversion,Res,elo_pre,elo_post
14183,8705,0,1,17,2018-05-20,Serie_A,Bologna,0,0,8,5,0.391913,-1,0,0,0.000000,0.0,895,879.815907
6873,8699,1,0,17,2018-05-20,Serie_A,Genoa,1,0,12,2,1.104570,-1,0,1,0.905330,0.0,938,920.955086
2634,3394,1,0,17,2018-05-20,La_Liga,Ath Bilbao,0,0,12,3,0.851602,-1,0,1,0.000000,0.0,1007,984.316860
9938,3394,0,0,17,2018-05-20,La_Liga,Espanol,1,1,11,4,1.087430,1,3,1,0.919599,1.0,1049,1072.831183
2635,3395,1,0,17,2018-05-20,La_Liga,Ath Madrid,2,1,8,4,1.468220,0,1,1,1.362194,0.5,1233,1219.268062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8437,1513,0,0,17,2018-05-13,EPL,Man City,1,0,13,2,0.765614,1,3,1,1.306141,1.0,1409,1413.956269
1133,1513,1,0,17,2018-05-13,EPL,Southampton,0,0,8,3,0.506624,-1,0,1,0.000000,0.0,936,932.097948
8436,1512,0,1,17,2018-05-13,EPL,Chelsea,0,0,6,2,0.843284,-3,0,0,0.000000,0.0,1171,1125.724066
1132,1512,1,1,17,2018-05-13,EPL,Newcastle,3,1,16,6,2.241970,3,3,3,1.338109,1.0,958,1004.670093


In [60]:
df = df.drop(['GD', 'abs_GD'], axis=1)
df['GD_Home'] = df.FTHG - df.FTAG
df['GD_Away'] = df.FTAG - df.FTHG
df['Pts_Home'] = np.where(df['FTHG'] == df['FTAG'], 1, np.where( df['FTHG'] >  df['FTAG'], 3, 0))
df['Pts_Away'] = np.where(df['FTAG'] == df['FTHG'], 1, np.where( df['FTAG'] >  df['FTHG'], 3, 0))
df['xPts_Home'] = np.where(round(df['home_xG']) == round(df['away_xG']), 1, np.where( round(df['home_xG']) >  round(df['away_xG']), 3, 0))
df['xPts_Away'] = np.where(round(df['away_xG']) == round(df['home_xG']), 1, np.where( round(df['away_xG']) >  round(df['home_xG']), 3, 0))
df['Conversion_Home'] = df.FTHG/df.home_xG
df['Conversion_Away'] = df.FTAG/df.away_xG
df['Home_win'] = (df['FTHG'] - df['FTAG'] > 0).astype(int)
df = df.rename({'HomeTeam':'Team_Home','AwayTeam':'Team_Away','FTHG':'FTG_Home','FTAG':'FTG_Away', 
               'HTHG':'HTG_Home','HTAG':'HTG_Away', 'HS':'S_Home','AS':'S_Away', 'HST':'ST_Home','AST':'ST_Away',
               'home_xG':'xG_Home','away_xG':'xG_Away', 'Home_elo_pre':'elo_pre_Home','Home_elo_post':'elo_post_Home',
               'Away_elo_pre':'elo_pre_Away','Away_elo_post':'elo_post_Away'}, axis = 1)
df.loc[:,'GameID'] = df.index
df = pd.wide_to_long(df, ['Team','FTG','HTG','S','ST','xG','GD','Pts','xPts','Conversion','Res','elo_pre','elo_post'], i = ['GameID'], j = 'isHome', sep = '_', suffix = r'\w+')
df = df.reset_index().sort_values(['Date','GameID'])
df.loc[:,'isHome'] = 1*(df.isHome == 'Home')


In [78]:
df.loc[:,'GP'] = df.groupby('Team').Pts.transform(lambda x : x.rolling(len(df), 1).count().shift(1, fill_value = 0))
df['GP_Season'] = (df['GP']) % np.where(df['Div'] == 'Bundesliga', 34, 38)
df.loc[:,'G_total'] = df.groupby('Team').FTG.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Pts_total'] = df.groupby('Team').Pts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Form3'] = df.groupby(['Team','Y']).Pts.transform(lambda x : x.rolling(3, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form5'] = df.groupby(['Team','Y']).Pts.transform(lambda x : x.rolling(5, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form7'] = df.groupby(['Team','Y']).Pts.transform(lambda x : x.rolling(7, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form3g'] = df.groupby(['Team','Y']).FTG.transform(lambda x : x.rolling(3, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form5g'] = df.groupby(['Team','Y']).FTG.transform(lambda x : x.rolling(5, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form7g'] = df.groupby(['Team','Y']).FTG.transform(lambda x : x.rolling(7, 1).sum().shift(1, fill_value = 0))
df.loc[:,'G_season'] = df.groupby(['Y','Team']).FTG.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Pts_season'] = df.groupby(['Y','Team']).Pts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Rank'] = df.groupby(['Y','Div', 'GP_Season'])['Pts_season'].rank("average", ascending=False)
df.loc[:,'GD_total'] = df.groupby('Team').GD.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xG_total'] = df.groupby('Team').xG.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xPts_total'] = df.groupby(['Team']).xPts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xPts_season'] = df.groupby(['Y','Team']).xPts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xRank'] = df.groupby(['Y','Div', 'GP_Season'])['xPts_season'].rank("average", ascending=False)
df['AGD'] = df['GD_total']/df['GP']
df['AxG'] = df['xG_total']/df['GP']
df['Reason'] = np.where((df['Rank'] >= 15) & (df['GP_Season'] >= 30), 1, 0)
df = df.fillna(0)

In [77]:
df2 = df.sort_values(by=['elo_post'], ascending = False)
df2 = df2.groupby(['Div', 'Team']).head(1)
df2 = df2.groupby(['Div']).head(3)
df2 = df2.sort_values(by=['Div','elo_post'], ascending = (True, False))
df2 = df2[['Div','Team','elo_post']]
df2

,Div,Team,elo_post
11223,Bundesliga,Bayern Munich,1431.181346
10987,Bundesliga,Dortmund,1275.239245
3675,Bundesliga,Hoffenheim,1222.223298
8582,EPL,Man City,1474.350450
1511,EPL,Liverpool,1429.354247
8381,EPL,Tottenham,1330.954050
2480,La_Liga,Barcelona,1470.607210
9587,La_Liga,Real Madrid,1384.346483
2586,La_Liga,Ath Madrid,1325.478099
5500,Ligue_1,Paris SG,1453.820457


,Div,Team,elo_post
11223,Bundesliga,Bayern Munich,1431.181346
10987,Bundesliga,Dortmund,1275.239245
3675,Bundesliga,Hoffenheim,1222.223298
8582,EPL,Man City,1474.350450
1511,EPL,Liverpool,1429.354247
8381,EPL,Tottenham,1330.954050
2480,La_Liga,Barcelona,1470.607210
9587,La_Liga,Real Madrid,1384.346483
2586,La_Liga,Ath Madrid,1325.478099
5500,Ligue_1,Paris SG,1453.820457


In [79]:
df1 = df.loc[(df.isHome == 1)]
df2 = df.loc[(df.isHome == 0)]
dfmerge = pd.merge(df1, df2, on=('GameID', 'Y', 'Date','Home_win','Div'), how='outer', suffixes=('_Home', '_Away'))
dfmerge.tail(6)

,GameID,isHome_Home,Home_win,Y,Date,Div,Team_Home,FTG_Home,HTG_Home,S_Home,ST_Home,xG_Home,GD_Home,Pts_Home,xPts_Home,Conversion_Home,Res_Home,elo_pre_Home,elo_post_Home,GP_Home,GP_Season_Home,G_total_Home,Pts_total_Home,Form3_Home,Form5_Home,Form7_Home,Form3g_Home,Form5g_Home,Form7g_Home,G_season_Home,Pts_season_Home,Rank_Home,GD_total_Home,xG_total_Home,xPts_total_Home,xPts_season_Home,xRank_Home,AGD_Home,AxG_Home,Reason_Home,isHome_Away,Team_Away,FTG_Away,HTG_Away,S_Away,ST_Away,xG_Away,GD_Away,Pts_Away,xPts_Away,Conversion_Away,Res_Away,elo_pre_Away,elo_post_Away,GP_Away,GP_Season_Away,G_total_Away,Pts_total_Away,Form3_Away,Form5_Away,Form7_Away,Form3g_Away,Form5g_Away,Form7g_Away,G_season_Away,Pts_season_Away,Rank_Away,GD_total_Away,xG_total_Away,xPts_total_Away,xPts_season_Away,xRank_Away,AGD_Away,AxG_Away,Reason_Away
7298,9080,1,0,18,2019-05-26,Serie_A,Fiorentina,0,0,5,5,0.296039,0,1,1,0.000000,0.5,942,936.859233,151,37,224,221,0,0,1,0,1,1,47,40,15.5,34,216.276911,238,57,8.0,0.225166,1.432297,1,0,Genoa,0,0,2,1,0.074391,0,1,1,0.000000,0.5,933,939.923495,151,37,155,160,2,3,4,3,4,5,39,37,18.0,-57,177.155361,174,39,13.0,-0.377483,1.173214,1
7299,9081,1,1,18,2019-05-26,Serie_A,Inter,2,0,20,15,2.743020,1,3,3,0.729123,1.0,1171,1177.977575,151,37,243,267,4,6,10,3,5,8,55,66,3.5,94,247.621969,276,75,3.0,0.622517,1.639881,0,0,Empoli,1,0,9,5,1.772750,-1,0,0,0.564095,0.0,979,972.819561,113,37,119,116,9,9,10,7,10,12,50,38,17.0,-59,123.218300,100,35,15.0,-0.522124,1.090427,1
7300,9082,1,1,18,2019-05-26,Serie_A,Roma,2,1,16,8,1.722810,1,3,3,1.160894,1.0,1194,1198.827356,151,37,298,307,5,9,15,3,7,9,64,63,6.0,144,279.722873,305,67,5.0,0.953642,1.852469,0,0,Parma,1,0,9,5,1.089380,-1,0,0,0.917953,0.0,925,921.420540,37,37,40,41,4,6,8,5,7,7,40,41,13.0,-19,40.009263,32,32,16.0,-0.513514,1.081331,0
7301,9083,1,1,18,2019-05-26,Serie_A,Sampdoria,2,0,10,3,0.686933,2,3,3,2.911492,1.0,974,1023.436422,151,37,211,192,2,2,5,4,5,7,58,50,9.0,-16,192.347462,164,50,9.0,-0.105960,1.273824,0,0,Juventus,0,0,6,1,0.487175,-2,0,0,0.000000,0.0,1333,1284.148215,151,37,308,367,2,6,9,2,5,8,70,90,1.0,209,254.994593,332,76,2.0,1.384106,1.688706,0
7302,9084,1,0,18,2019-05-26,Serie_A,Spal,2,1,7,4,0.433984,-1,0,0,4.608465,0.0,1002,986.486582,75,37,81,80,3,7,10,7,12,15,42,42,11.0,-31,77.463586,70,39,13.0,-0.413333,1.032848,0,0,Milan,3,2,16,8,1.784600,1,3,3,1.681049,1.0,1168,1184.506517,151,37,214,249,9,10,13,5,6,8,52,65,5.0,50,214.834954,238,59,7.0,0.331126,1.422748,0
7303,9085,1,1,18,2019-05-26,Serie_A,Torino,3,0,9,7,2.398040,2,3,3,1.251022,1.0,1140,1158.974142,151,37,226,212,4,10,12,5,8,9,49,60,7.0,23,197.482190,189,45,10.0,0.152318,1.307829,0,0,Lazio,1,0,9,4,1.179200,-2,0,0,0.848033,0.0,1097,1079.098431,151,37,270,255,4,7,10,6,9,11,55,59,8.0,75,247.187643,255,61,6.0,0.496689,1.637004,0


,GameID,isHome_Home,Home_win,Y,Date,Div,Team_Home,FTG_Home,HTG_Home,S_Home,ST_Home,xG_Home,GD_Home,Pts_Home,xPts_Home,Conversion_Home,Res_Home,elo_pre_Home,elo_post_Home,GP_Home,GP_Season_Home,G_total_Home,Pts_total_Home,Form3_Home,Form5_Home,Form7_Home,Form3g_Home,Form5g_Home,Form7g_Home,G_season_Home,Pts_season_Home,Rank_Home,GD_total_Home,xG_total_Home,xPts_total_Home,xPts_season_Home,xRank_Home,AGD_Home,AxG_Home,Reason_Home,isHome_Away,Team_Away,FTG_Away,HTG_Away,S_Away,ST_Away,xG_Away,GD_Away,Pts_Away,xPts_Away,Conversion_Away,Res_Away,elo_pre_Away,elo_post_Away,GP_Away,GP_Season_Away,G_total_Away,Pts_total_Away,Form3_Away,Form5_Away,Form7_Away,Form3g_Away,Form5g_Away,Form7g_Away,G_season_Away,Pts_season_Away,Rank_Away,GD_total_Away,xG_total_Away,xPts_total_Away,xPts_season_Away,xRank_Away,AGD_Away,AxG_Away,Reason_Away
7298,9080,1,0,18,2019-05-26,Serie_A,Fiorentina,0,0,5,5,0.296039,0,1,1,0.000000,0.5,942,936.859233,151,37,224,221,0,0,1,0,1,1,47,40,15.5,34,216.276911,238,57,8.0,0.225166,1.432297,1,0,Genoa,0,0,2,1,0.074391,0,1,1,0.000000,0.5,933,939.923495,151,37,155,160,2,3,4,3,4,5,39,37,18.0,-57,177.155361,174,39,13.0,-0.377483,1.173214,1
7299,9081,1,1,18,2019-05-26,Serie_A,Inter,2,0,20,15,2.743020,1,3,3,0.729123,1.0,1171,1177.977575,151,37,243,267,4,6,10,3,5,8,55,66,3.5,94,247.621969,276,75,3.0,0.622517,1.639881,0,0,Empoli,1,0,9,5,1.772750,-1,0,0,0.564095,0.0,979,972.819561,113,37,119,116,9,9,10,7,10,12,50,38,17.0,-59,123.218300,100,35,15.0,-0.522124,1.090427,1
7300,9082,1,1,18,2019-05-26,Serie_A,Roma,2,1,16,8,1.722810,1,3,3,1.160894,1.0,1194,1198.827356,151,37,298,307,5,9,15,3,7,9,64,63,6.0,144,279.722873,305,67,5.0,0.953642,1.852469,0,0,Parma,1,0,9,5,1.089380,-1,0,0,0.917953,0.0,925,921.420540,37,37,40,41,4,6,8,5,7,7,40,41,13.0,-19,40.009263,32,32,16.0,-0.513514,1.081331,0
7301,9083,1,1,18,2019-05-26,Serie_A,Sampdoria,2,0,10,3,0.686933,2,3,3,2.911492,1.0,974,1023.436422,151,37,211,192,2,2,5,4,5,7,58,50,9.0,-16,192.347462,164,50,9.0,-0.105960,1.273824,0,0,Juventus,0,0,6,1,0.487175,-2,0,0,0.000000,0.0,1333,1284.148215,151,37,308,367,2,6,9,2,5,8,70,90,1.0,209,254.994593,332,76,2.0,1.384106,1.688706,0
7302,9084,1,0,18,2019-05-26,Serie_A,Spal,2,1,7,4,0.433984,-1,0,0,4.608465,0.0,1002,986.486582,75,37,81,80,3,7,10,7,12,15,42,42,11.0,-31,77.463586,70,39,13.0,-0.413333,1.032848,0,0,Milan,3,2,16,8,1.784600,1,3,3,1.681049,1.0,1168,1184.506517,151,37,214,249,9,10,13,5,6,8,52,65,5.0,50,214.834954,238,59,7.0,0.331126,1.422748,0
7303,9085,1,1,18,2019-05-26,Serie_A,Torino,3,0,9,7,2.398040,2,3,3,1.251022,1.0,1140,1158.974142,151,37,226,212,4,10,12,5,8,9,49,60,7.0,23,197.482190,189,45,10.0,0.152318,1.307829,0,0,Lazio,1,0,9,4,1.179200,-2,0,0,0.848033,0.0,1097,1079.098431,151,37,270,255,4,7,10,6,9,11,55,59,8.0,75,247.187643,255,61,6.0,0.496689,1.637004,0


In [84]:
dfmerge['ELO_Diff'] = dfmerge.elo_pre_Home - dfmerge.elo_pre_Away

dfhw = dfmerge.loc[dfmerge['Y'] < 18]
df18 = dfmerge.loc[dfmerge['Y'] == 18]
result = smf.glm('Home_win ~ AGD_Home + AGD_Away + Form5g_Home + Form5g_Away + AxG_Home + xPts_season_Home + Rank_Home + xRank_Away + Reason_Home + ELO_Diff', data = dfhw, family = sm.families.Binomial()).fit()
result.summary(), print(brier_score_loss(df18.Home_win, result.predict(df18)))

0.21396717477854457


(<class 'statsmodels.iolib.summary.Summary'>
 """
                  Generalized Linear Model Regression Results                  
 Dep. Variable:               Home_win   No. Observations:                 5478
 Model:                            GLM   Df Residuals:                     5467
 Model Family:                Binomial   Df Model:                           10
 Link Function:                  logit   Scale:                          1.0000
 Method:                          IRLS   Log-Likelihood:                -3391.1
 Date:                Wed, 24 Feb 2021   Deviance:                       6782.1
 Time:                        21:36:20   Pearson chi2:                 5.44e+03
 No. Iterations:                     4                                         
 Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
 ---------------------------------------------------------------

0.21396717477854457


(<class 'statsmodels.iolib.summary.Summary'>
 """
                  Generalized Linear Model Regression Results                  
 Dep. Variable:               Home_win   No. Observations:                 5478
 Model:                            GLM   Df Residuals:                     5467
 Model Family:                Binomial   Df Model:                           10
 Link Function:                  logit   Scale:                          1.0000
 Method:                          IRLS   Log-Likelihood:                -3391.1
 Date:                Wed, 24 Feb 2021   Deviance:                       6782.1
 Time:                        21:36:20   Pearson chi2:                 5.44e+03
 No. Iterations:                     4                                         
 Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
 ---------------------------------------------------------------